# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [2]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df['col2'] = [2,2,2,2]
# df

## 2. Deleting a row in a DataFrame

In [3]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df.drop('d',inplace=True)
df

,col1
a,1
b,2
c,3


## 3. Creating a DataFrame from a few Series

In [4]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))

exc = pd.DataFrame(zip(ser_1,ser_2,ser_3),columns=['ser_1','ser_2','ser_3'])
exc

,ser_1,ser_2,ser_3
0,0.261356,-0.076254,0.457411
1,0.222123,0.587254,-1.102306
2,1.505329,-0.531581,-1.515866
3,-0.069592,-0.362820,-0.030461
4,-0.276663,-0.593932,1.249773
5,0.139426,0.843569,0.223945


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [5]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])

print df
df['col_2']

      col_1  col_2 col_3
obs1   0.12    0.9   NaN
obs2   7.00    9.0   NaN
obs3  45.00   34.0   NaN
obs4  10.00   11.0   NaN


obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [6]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.loc['obs3']

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [7]:
# using the same DataFrame, index into into its first row
df.iloc[0]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [8]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


## 2. How to load the training and testing subsets

In [9]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0)
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0)

In [10]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [43]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [14]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [15]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [16]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [17]:
print 'RMSE for my estimate function: %s' % evaluate(my_estimate_func)

RMSE for my estimate function: 1.23237195265


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [49]:
movielens_train.head(2)

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False


In [19]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    # first, index into all ratings of this movie
    # second, compute the mean of those ratings
    # 
    if movie_id in movielens_train.movie_id.tolist():
        ratings_all = []
        for i in range(len(movielens_train)):
            if movielens_train.movie_id.iloc[i] == movie_id:
                ratings_all.append(movielens_train.rating.iloc[i])
        return np.mean(ratings_all)
    else:
        print 'movie_id not found.'

    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

4.0

# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [25]:
movielens_train.head(1)

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False


In [21]:
users.head(1)

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067


In [36]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [57]:
# 4653 2648 2579    Horror
genre = movies[movies.movie_id==2648].genres.values[0]
print genre
a = movielens_train[(movielens_train.genres==genre) & (movielens_train.user_id!=4653)].rating
b = movielens_train[(movielens_train.genres==genre) 
                    & (movielens_train.user_id==4653)
                   & (movielens_train.movie_id!=2648)].rating
w = np.array([0.5,0.5])
print w*a.mean()

Horror
3.01863354037


In [63]:
%%time

class CollabGenreReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by same genres of other users. """
        genre = movies[movies.movie_id==movie_id].genres.values[0]

        # get other user ratings of this genre
        their_ratings = movielens_train[(movielens_train.genres==genre) & (movielens_train.user_id!=user_id)].rating
        self_rating = movielens_train[(movielens_train.genres==genre) & 
                                      (movielens_train.user_id==user_id) &
                                     (movielens_train.movie_id!=movie_id)].rating
        if their_ratings.empty:
            return 3.0
        elif self_rating.empty:
            return their_ratings.mean()
        else:
            weights = np.array([0.6,0.4])
            ratings = np.array([self_rating.mean(),their_ratings.mean()])
            return weights.dot(ratings.T)
        
reco = CollabGenreReco()
print 'RMSE for CollabPearsonReco: %s' % evaluate(reco.estimate)

RMSE for CollabPearsonReco: 1.0809497458
CPU times: user 10.2 s, sys: 52.5 ms, total: 10.2 s
Wall time: 10.3 s


# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [64]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [66]:
class CollabSimReco:
    """ Collaborative filtering using a custom sim(u,u'). """
    def __init__(self,method='cosine'):
        self.method = method.lower()
    
    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        if self.method == 'cosine':
            sims = their_profiles.apply(lambda profile: cosine(profile, user_profile), axis=0)
        elif self.method == 'euclidean':
            sims = their_profiles.apply(lambda profile: euclidean(profile,user_profile), axis=0)
        else:
            print 'Invalid method'
            return
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabSimReco('cosine')
reco.learn()
print 'RMSE for CollabSimReco: %s' % evaluate(reco.estimate)

RMSE for CollabPearsonReco: 1.13439921696


In [69]:
reco = CollabSimReco('euclidean')
reco.learn()
print 'RMSE for CollabSimReco: %s' % evaluate(reco.estimate)

RMSE for CollabPearsonReco: 1.13439921696
